In [3]:
import pandas, numpy

In [4]:
file_directory = "/Users/kja11/OneDrive - Menntaský/PostDoc_Hypothermia/in_silico/Python/"

In [11]:
proteomics_path = file_directory+'3) output/Proteomics/signif_linregress_proteins.txt'
RNAseq_path = file_directory+'3) output/RNAseq/allsignif_genes_HEK293.txt'

In [16]:
with open(proteomics_path) as f:
    proteomics = f.read().splitlines()

with open(RNAseq_path) as f:
    RNAseq = f.read().splitlines()

In [17]:
# Convert lists to sets and find the intersection
common_elements = set(proteomics).intersection(RNAseq)

# Convert the result back to a list if needed
common_elements = list(common_elements)

print(common_elements)

['CHORDC1', 'CIRBP', 'SRSF5', 'RBM3', 'HNRNPDL']
